In [1]:
!pip install pandas trl accelerate transformers datasets evaluate rouge_score -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 6.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 23.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.9 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account a

In [2]:
import trl
print(trl.__version__)  # 0.4.7
import transformers
print(transformers.__version__)  # 4.30.2

0.17.0
4.51.1


In [3]:
import pandas as pd
import csv

# Đọc dữ liệu an toàn, chỉ lấy 2 cột
df = pd.read_csv(
    "/kaggle/input/summary-vietnamese/data_summary.csv",
    usecols=["Text", "Summary"],         # chỉ lấy 2 cột chính
    quoting=csv.QUOTE_MINIMAL,           # giữ nguyên văn bản dài
    encoding="utf-8",
    on_bad_lines="skip",                 # bỏ dòng lỗi
    engine="python"
)
print(f"Tổng dòng trước xử lý: {len(df)}")

# Làm sạch dữ liệu
df = df.dropna()
df = df.drop_duplicates()
df = df[df["Text"].str.len() > 50]
df = df[df["Summary"].str.split().apply(len) >= 5]
df = df.reset_index(drop=True)

print(f"Tổng dòng sau xử lý: {len(df)}")
df_first_100 = df[:100]
df_first_100.head()


Tổng dòng trước xử lý: 102680
Tổng dòng sau xử lý: 89333


,Summary,Text
0,Ngay khi bắt tay ai đó có thể chúng ta đã cảm ...,Tổng thống Mỹ Donald Trump và Tổng thống Pháp ...
1,"Rời khỏi Việt Nam vào 10/2016, ông Vũ Đình Duy...",Vũ Đình Duy hồi tháng 5 xuất hiện trước tòa tạ...
2,Thủ tướng Iraq xác nhận với BBC rằng Syria khô...,Ông Maliki xác nhận các vụ không kích của Syri...
3,Người Việt Nam biết rất ít về ông Trump nhưng ...,Tiến sỹ Vũ Cao Phan tin rằng nếu được tiến hàn...
4,Bốn khoa học gia người Mỹ từng đoạt giải Nobel...,GS Jack Steinberger trao đổi với GS Lê Kim Ngọ...


In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "vinai/bartpho-syllable"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/897 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

dict.txt:   0%|          | 0.00/360k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

2025-05-03 16:04:29.823857: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746288270.017624      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746288270.070920      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


pytorch_model.bin:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

In [5]:
from datasets import Dataset

# Giả sử df đã có cột 'Text' và 'Summary'
dataset = Dataset.from_pandas(df_first_100[["Text", "Summary"]])
dataset = dataset.train_test_split(test_size=0.05, seed=42)


In [6]:
max_input_length = 512
max_target_length = 64

def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["Text"], max_length=max_input_length, truncation=True, padding="max_length"
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["Summary"], max_length=max_target_length, truncation=True, padding="max_length"
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/95 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [7]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [8]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./bartpho_summary",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    save_total_limit=2,
    predict_with_generate=True,
    logging_dir="./logs",
    fp16=True,                        # Mixed precision
    gradient_accumulation_steps=2,   # Giả lập batch size lớn hơn
    logging_steps=100,
    eval_steps=100,
    save_steps=500
)



trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipykernel_31/1999921058.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss


TrainOutput(global_step=3, training_loss=11.123695373535156, metrics={'train_runtime': 14.3542, 'train_samples_per_second': 6.618, 'train_steps_per_second': 0.209, 'total_flos': 102938663976960.0, 'train_loss': 11.123695373535156, 'epoch': 1.0})

In [9]:
def summarize(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=64)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test mẫu
sample = df["Text"].iloc[0]
print("Input:", sample)
print("Summary:", summarize(sample))

Input: Tổng thống Mỹ Donald Trump và Tổng thống Pháp Emmanuel Macron đi cạnh nhau, nắm chặt tay nhau. Loại thuốc biến con người thành kẻ sát nhân Nghệ thuật tặng quà làm vừa lòng người nhận Họ giữ tay trong tay giống như kẹp dính vậy, kèm theo động tác kéo mạnh, vỗ vai nhau trong vòng 29 giây đồng hồ trước bao ống kính của truyền thông thế giới. Diễn ra trong cuộc diễu hành nhân ngày Quốc khánh Pháp - kỷ niệm ngày phá ngục Bastille, một bước ngoặt của cuộc cách mạng Pháp - lẽ ra đó nên là khoảnh khắc đoàn kết giữa các nhà lãnh đạo của hai quốc gia vĩ đại. Vậy nhưng nó đã trở thành một trong những cái bắt tay khiên cưỡng, dùng dằng lâu nhất trong thời gian gần đây. Người ta không thể nghe thấy những gì họ nói vì họ gần như ép sát vào nhau, nhưng ngôn ngữ cơ thể họ đã lấn át đi lời nói. Một nghi thức chào hỏi hoàn hảo trong tình huống này có thể bao gồm nhiều giao tiếp bằng mắt và cử chỉ cơ thể tích cực. "Cách tốt nhất nên thể hiện là tạo ra hình ảnh bình đẳng giữa hai bên," bà Sanda D

In [26]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import evaluate
import pandas as pd
from datasets import Dataset
from torch.utils.data import DataLoader
from torch.optim import AdamW
import numpy as np

# Load model and tokenizer
model_name = "vinai/bartpho-syllable"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Load data from DataFrame (assuming df exists)
dataset = Dataset.from_dict({
    "query": df["Text"].tolist()[:50],
    "reference": df["Summary"].tolist()[:50]
})

# Load ROUGE metric
rouge = evaluate.load("rouge")

def compute_rouge_score(generated, reference):
    score = rouge.compute(predictions=[generated], references=[reference])
    return score["rougeL"]

# Tokenize dataset
def tokenize_function(examples):
    inputs = tokenizer(examples["query"], max_length=512, padding="max_length", truncation=True, return_tensors="pt")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["reference"], max_length=64, padding="max_length", truncation=True, return_tensors="pt")
    return {
        "input_ids": inputs["input_ids"].squeeze(),
        "attention_mask": inputs["attention_mask"].squeeze(),
        "labels": labels["input_ids"].squeeze()
    }

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["query", "reference"])
tokenized_dataset.set_format("torch")

# Create DataLoader
dataloader = DataLoader(tokenized_dataset, batch_size=2, shuffle=True)

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training loop
model.train()
num_epochs = 3

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    total_loss = 0
    for batch in dataloader:
        # Move batch to device
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        logits = outputs.logits

        # Generate text for ROUGE-based reward
        generated_ids = model.generate(input_ids, max_new_tokens=64, num_beams=4)
        generated_texts = [tokenizer.decode(gid, skip_special_tokens=True) for gid in generated_ids]
        reference_texts = [tokenizer.decode(lb, skip_special_tokens=True) for lb in labels]

        # Compute ROUGE-based reward
        rouge_scores = [compute_rouge_score(gen, ref) for gen, ref in zip(generated_texts, reference_texts)]
        rouge_tensor = torch.tensor(rouge_scores, dtype=torch.float32, device=device)

        # Combine model loss with ROUGE reward (use negative ROUGE as a penalty)
        loss = outputs.loss - 0.1 * torch.mean(rouge_tensor)  # Adjust weight as needed
        total_loss += loss.item()

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Logging
        for i, (prompt, gen, ref, score) in enumerate(zip(batch["input_ids"], generated_texts, reference_texts, rouge_scores)):
            print(f"🟩 ROUGE-L: {score:.4f}")
            print(f"📌 Prompt: {tokenizer.decode(prompt, skip_special_tokens=True)[:50]}...")
            print(f"📝 Generated: {gen}")
            print(f"📋 Reference: {ref}")
            print("-" * 50)

    avg_loss = total_loss / len(dataloader)
    print(f"Average Loss: {avg_loss:.4f}\n")

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Epoch 1/3
🟩 ROUGE-L: 0.0000
📌 Prompt: Chưa biết tới khi nào người dân TQ mới biết được l...
📝 Generated: " """" "" "" " "" "" "" "" "" "" "" """ "" "" "" "" "" "" "" "" "" "" "" "" "" "" """"" ""
📋 Reference: Một quan chức điều tra an toàn của Trung Quốc đang làm dư luận khó hiểu sau khi đưa ra những lời bình luận trái ngược nhau về nguyên nhân của vụ tai nạn tàu cao tốc hồi tháng Bảy.
--------------------------------------------------
🟩 ROUGE-L: 0.0690
📌 Prompt: Trên sân Estadio Nacional của Brasilia, Brazil gặ...
📝 Generated: Trên sang mùm
📋 Reference: Trước khi bước vào lượt đá cuối cùng vòng đấu bảng, có ba đội ở bảng A cùng có cơ hội giành vé đi tiếp là Brazil, Mexico và Croatia, trong lúc Cameroon đã có sẵn vé rời World Cup.
--------------------------------------------------
🟩 ROUGE-L: 0.1071
📌 Prompt: Những người thuộc phe đối lập dùng cây chặn đườ...
📝 Generated: Những hữu
📋 Reference: Người đứng đầu Tổ chức Các quốc gia châu Mỹ (OAS), ông Luis Almagro, cảnh báo tình hình

In [27]:
from evaluate import load

rouge = load("rouge")

preds = ["tóm tắt sinh ra"]
refs = ["tóm tắt chuẩn"]

rouge.compute(predictions=preds, references=refs)

{'rouge1': 0.6666666666666666,
 'rouge2': 0.6,
 'rougeL': 0.6666666666666666,
 'rougeLsum': 0.6666666666666666}

In [30]:
# Save the fine-tuned model
model.save_pretrained("./finetuned_bartpho")
tokenizer.save_pretrained("./finetuned_bartpho")

('./finetuned_bartpho/tokenizer_config.json',
 './finetuned_bartpho/special_tokens_map.json',
 './finetuned_bartpho/sentencepiece.bpe.model',
 './finetuned_bartpho/dict.txt',
 './finetuned_bartpho/added_tokens.json')

In [32]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("./finetuned_bartpho")

def summarize(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=64)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Thử nghiệm
input_text = "Sau sáp nhập, cấp tỉnh dự kiến giảm 18.440 biên chế cán bộ, công chức và cấp xã giảm 110.780 biên chế so với số lượng được cấp có thẩm quyền giao năm 2022. Sáng 3/5, Tổng Bí thư Tô Lâm chủ trì họp Bộ Chính trị, Ban Bí thư, cho ý kiến về tình hình, tiến độ sắp xếp tổ chức bộ máy và đơn vị hành chính các cấp. Theo báo cáo tại cuộc họp, các địa phương hoàn thành lấy ý kiến nhân dân về các đề án sáp nhập tỉnh và xã với tỷ lệ đồng thuận trung bình 96%. Tất cả HĐND cấp huyện, xã và tỉnh đã ban hành nghị quyết thông qua các đề án với tỷ lệ đồng thuận đa số đạt 100%."
print(summarize(input_text))

Sau sáp nhập, cấp tỉnh dự kiến giảm 18.440 biên chế cán bộ, công chức và cấp xã giảm 110.780 biên chế so với số lượng được cấp có thẩm quyền giao năm 2022. Sáng 3/5, Tổng Bí thư Tô Lâm chủ trì họp Bộ Chính trị, Ban Bí thư, cho
